# Cycombine for missing marker imputation

In [1]:
library(dplyr)
library(cyCombine)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
min_max_scale <- function(data, columns) {
  columns <- intersect(columns, colnames(data))
  
  # scale each column
  for (col in columns) {
    min_val <- min(data[[col]], na.rm = TRUE)
    max_val <- max(data[[col]], na.rm = TRUE)
    
    # apply scaling
    data[[col]] <- (data[[col]] - min_val) / (max_val - min_val)
  }
  
  return(data)
}

In [5]:
# load data
expr  <- read.csv('/home/projects/amit/floriani/Lab/PROJECTS/FlowVI/data/2024-01-16_model_eval_multi_batch/2025-08-26_aurora_imputation_expression_scaled.csv')[,-1]
md  <- read.csv('/home/projects/amit/floriani/Lab/PROJECTS/FlowVI/data/2024-01-16_model_eval_multi_batch/2024-07-05_aurora_imputation_obs.csv')[, -1]

markers  <- colnames(expr)

In [ ]:
expr[,'batch']  <- md$synth_batch
expr[,'sample']  <- md$sample_id

batch_mask  <- '1'
imputed_list  <- list()

for (mask_marker in markers){
    BB_marker  <- setdiff(markers, mask_marker)

    labels  <- expr %>% 
        normalize(markers = BB_marker, norm_method = 'scale') %>%
        create_som(markers = BB_marker, rlen = 10)

    corrected_expr <- expr %>%
        correct_data(label = labels, markers = BB_marker)

    cor_expr_batch0 <- corrected_expr %>%
        filter(batch == '0')

    cor_expr_batch1 <- corrected_expr %>%
        filter(batch == '1')

    imputed <- impute_across_panels(dataset1 = cor_expr_batch0, 
                                    dataset2 = cor_expr_batch1,
                                    overlap_channels = BB_marker, 
                                    impute_channels1 = NULL,
                                    impute_channels2 = mask_marker)

    imputed_list[[mask_marker]]  <- imputed$dataset2[mask_marker]
    }

In [8]:
imputed_df  <- bind_cols(imputed_list)

In [9]:
write.csv(imputed_df, file = '/home/projects/amit/floriani/Lab/PROJECTS/FlowVI/data/2024-01-16_model_eval_multi_batch/2025-08-26_aurora_imputation_cycombine_imputed.csv')